In [40]:
import pandas as pd

df = pd.read_csv("new_netflix_titles.csv")

In [51]:
%load_ext autoreload
%autoreload 2
import selenium
import netflix_collection as helper
import creds
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import creds
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
import time

success_dict ={}

def initialize_driver():
    options = webdriver.ChromeOptions()
    proto = "TCP"
    #options = webdriver.ChromeOptions()
    #options.add_argument("--headless")
    #print(os.path.exists())
    options.add_argument("--disable-logging")
    options.add_argument("--window-size=2000,3555") # Needs to be big enough to get all the resolutions
    if (proto == "TCP"):
        options.add_argument("--disable-quic")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--no-sandbox")

    #stopthreads =False
    #th = threading.Thread(target=capture_live_packets, args=(url,intface,lambda: stopthreads,))


    driver = webdriver.Chrome(chrome_options=options,executable_path="/usr/bin/chromedriver_linux64/chromedriver")
    return driver

def check_if_url_valid(driver,page_url):
    start_time= time.time()
    retries = 0
    while(True):
        video_player_length = driver.execute_script(
        r"""return document.getElementsByTagName("video").length""")

        if (video_player_length>0):
            return True
        
        try:
            error_line = driver.find_element("xpath", "/html/body/div[1]/div/div/div[1]/div/div/div[2]/div/div[2]/span").text
        except:
            error_line = "Unsure"
        if ("Error Code" in error_line):
            return False
        #print(error_line)
        time.sleep(0.5)

        current_time = time.time()
        if (current_time - start_time > 5):
            driver.refresh()
            retries = retries +1
            start_time = current_time





driver = initialize_driver()
collection_start = time.time()
helper.log_in(driver)
url = "https://www.netflix.com/watch"
df = pd.read_csv("new_netflix_titles.csv")
for idx,row in df.iterrows():
    if (row["url_valid"] == True or row["url_valid"] == False):
        print(f"skipping_{row['title']}")
        continue


    if (time.time() - collection_start > 600):
        driver.quit()
        driver = initialize_driver()
        helper.log_in(driver)
        collection_start = time.time()
    
    formatted_url = f"{url}/{row['show_id']}"
    page_url = driver.get(formatted_url)
    is_valid = check_if_url_valid(driver,formatted_url)
    df.at[idx, 'url_valid'] = is_valid
    df.to_csv("new_netflix_titles.csv")
    time.sleep(3)


        


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/tmp/ipykernel_1824934/2831916494.py:34: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_options=options,executable_path="/usr/bin/chromedriver_linux64/chromedriver")
/tmp/ipykernel_1824934/2831916494.py:34: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options,executable_path="/usr/bin/chromedriver_linux64/chromedriver")


LOGIN SUCCESSFUL!
skipping_Norm of the North: King Sized Adventure
skipping_Jandino: Whatever it Takes
skipping_Transformers Prime
skipping_Transformers: Robots in Disguise
skipping_#realityhigh
skipping_Apaches
skipping_Automata
skipping_Fabrizio Copano: Solo pienso en mi
skipping_Fire Chasers
skipping_Good People
skipping_Joaquín Reyes: Una y no más
skipping_Kidnapping Mr. Heineken
skipping_Krish Trish and Baltiboy
skipping_Krish Trish and Baltiboy: Battle of Wits
skipping_Krish Trish and Baltiboy: Best Friends Forever
skipping_Krish Trish and Baltiboy: Comics of India
skipping_Krish Trish and Baltiboy: Oversmartness Never Pays
skipping_Krish Trish and Baltiboy: Part II
skipping_Krish Trish and Baltiboy: The Greatest Trick
skipping_Love
skipping_Manhattan Romance
skipping_Moonwalkers
skipping_Rolling Papers
skipping_Stonehearst Asylum
skipping_The Runner
skipping_6 Years
skipping_Castle of Stars
skipping_City of Joy
skipping_First and Last
skipping_Laddaland
skipping_Next Gen
skippin

/tmp/ipykernel_1824934/2831916494.py:34: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_options=options,executable_path="/usr/bin/chromedriver_linux64/chromedriver")
/tmp/ipykernel_1824934/2831916494.py:34: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=options,executable_path="/usr/bin/chromedriver_linux64/chromedriver")


LOGIN SUCCESSFUL!


KeyboardInterrupt: 

In [33]:
success_dict

{}

In [42]:
row

Unnamed: 0                                                    442
show_id                                                  80215730
type                                                      TV Show
title                               Legend Quest: Masters of Myth
director                                                      NaN
cast            Johnny Rose, Annemarie Blanco, Paul Tei, Oscar...
country                                                    Mexico
date_added                                        October 5, 2019
release_year                                                 2019
rating                                                      TV-Y7
duration                                                 1 Season
listed_in                                                Kids' TV
description     When mythical creatures come to life, it's up ...
url_valid                                                     NaN
Name: 442, dtype: object